# ⚖️ 第二章 练习 2: 逆概率加权 (Inverse Probability Weighting, IPW)

---

## 从「丢弃样本」到「重新加权」

在上一个练习中，我们学习了 PSM。它很直观，但有一个问题：

> 很多控制组的样本没有被匹配上，被白白丢弃了！

有没有办法利用所有样本呢？有的！这就是**逆概率加权 (IPW)**。

### 一个投票的类比 🗳️

想象一下选举民调：

你想知道全国人民对某候选人的支持率，但你只能调查到一部分人。问题是：
- 年轻人更愿意参与调查（响应率高）
- 老年人不太愿意参与（响应率低）

如果直接用样本计算支持率，会偏向年轻人的观点。

**解决方案**: 给每个人一个「权重」！
- 年轻人权重小（因为采样时已经被过度代表了）
- 老年人权重大（因为采样时被低估了）

这样加权平均后，就能还原真实的支持率！

**IPW 的思想完全一样**：给那些「不太可能接受处理但接受了」的人更大权重，给那些「很可能接受处理也接受了」的人更小权重。

---

## 📚 本节学习目标

1. 理解 IPW 的核心思想：创造「伪总体」
2. 实现 IPW 权重计算
3. 使用加权方法估计 ATE
4. 理解极端权重的问题和解决方法
5. 诊断权重分布和有效样本量

In [ ]:
# 导入必要的库
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from typing import Tuple, Optional

# 设置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False
sns.set_style('whitegrid')

print("✅ 环境准备完毕！")

---

## 🌟 Part 1: IPW 的核心思想

### 为什么需要加权？

在观测数据中，处理组和控制组的特征分布不同。

**例子**: 研究「读研」对「收入」的影响
- 读研的人往往本科成绩更好
- 本科成绩好的人不读研收入也高

如果直接比较，会高估读研的效果。

### IPW 如何工作？

**核心思想**: 通过加权，让处理组和控制组「看起来」像是随机分配的。

权重公式：

$$w_i = \frac{T_i}{e(X_i)} + \frac{1-T_i}{1-e(X_i)}$$

其中 $e(X_i) = P(T=1|X_i)$ 是倾向得分。

### 直觉理解

| 情况 | 权重 | 为什么？ |
|-----|------|----------|
| 很可能被处理，也被处理了 | 小 | 这类人在处理组中已经很多了 |
| 不太可能被处理，但被处理了 | 大 | 这类人在处理组中很稀少，代表了更多人 |
| 很可能被处理，但没被处理 | 大 | 这类人在控制组中很稀少 |
| 不太可能被处理，也没被处理 | 小 | 这类人在控制组中已经很多了 |

### 🧮 数学推导

IPW 估计器的核心公式：

$$\hat{E}[Y(1)] = \frac{1}{n}\sum_{i=1}^{n} \frac{T_i Y_i}{e(X_i)}$$

$$\hat{E}[Y(0)] = \frac{1}{n}\sum_{i=1}^{n} \frac{(1-T_i) Y_i}{1-e(X_i)}$$

$$\widehat{ATE} = \hat{E}[Y(1)] - \hat{E}[Y(0)]$$

为什么这个公式有效？

因为 $E\left[\frac{TY}{e(X)}\right] = E\left[E\left[\frac{TY}{e(X)}\Big|X\right]\right] = E\left[\frac{E[TY|X]}{e(X)}\right] = E\left[\frac{e(X) \cdot E[Y|X,T=1]}{e(X)}\right] = E[Y(1)]$

---

## 📐 IPW 无偏性证明

**定理**: 如果满足无混淆假设和正值假设，IPW 估计量是 ATE 的无偏估计。

**证明**:

我们需要证明 $E\left[\frac{T \cdot Y}{e(X)}\right] = E[Y(1)]$

**Step 1**: 对 X 条件化
$$E\left[\frac{T \cdot Y}{e(X)}\right] = E\left[E\left[\frac{T \cdot Y}{e(X)} \Big| X\right]\right]$$

**Step 2**: 在给定 X 的条件下，$e(X)$ 是常数，可以提出来
$$= E\left[\frac{1}{e(X)} \cdot E[T \cdot Y | X]\right]$$

**Step 3**: 分解 $E[T \cdot Y | X]$
$$E[T \cdot Y | X] = E[T \cdot Y | X, T=1] \cdot P(T=1|X) + E[T \cdot Y | X, T=0] \cdot P(T=0|X)$$

由于 $T=0$ 时 $T \cdot Y = 0$，所以：
$$= E[Y | X, T=1] \cdot P(T=1|X) = E[Y | X, T=1] \cdot e(X)$$

**Step 4**: 代回原式
$$E\left[\frac{1}{e(X)} \cdot E[T \cdot Y | X]\right] = E\left[\frac{1}{e(X)} \cdot E[Y|X,T=1] \cdot e(X)\right] = E[E[Y|X,T=1]]$$

**Step 5**: 应用无混淆假设 $Y(1) \perp T | X$
$$E[Y|X,T=1] = E[Y(1)|X,T=1] = E[Y(1)|X]$$

**Step 6**: 结论
$$E\left[\frac{T \cdot Y}{e(X)}\right] = E[E[Y(1)|X]] = E[Y(1)]$$

同理可证 $E\left[\frac{(1-T) \cdot Y}{1-e(X)}\right] = E[Y(0)]$

因此：$$E[\widehat{ATE}_{IPW}] = E[Y(1)] - E[Y(0)] = ATE$$

**关键假设**:
1. **无混淆**: $(Y(0), Y(1)) \perp T | X$
2. **正值**: $0 < e(X) < 1$ （否则分母为 0）

---

## 📊 Horvitz-Thompson 估计量的视角

IPW 实际上是 **Horvitz-Thompson 估计量**在因果推断中的应用。

在抽样理论中，如果个体 i 被抽中的概率是 $\pi_i$，那么总体均值的无偏估计是：

$$\hat{\mu} = \frac{1}{N}\sum_{i \in \text{sample}} \frac{y_i}{\pi_i}$$

在因果推断中：
- 处理组的"抽样概率"是 $e(X_i)$
- 控制组的"抽样概率"是 $1-e(X_i)$

所以 IPW 就是在用 HT 估计量估计 $E[Y(1)]$ 和 $E[Y(0)]$！

---

---

## 🔬 Part 2: 实现 IPW

In [ ]:
def generate_ipw_data(n: int = 2000, confounding_strength: float = 1.5, seed: int = 42):
    """
    生成用于 IPW 实验的数据
    
    真实 ATE = 2
    """
    np.random.seed(seed)
    
    # 生成协变量
    X1 = np.random.randn(n)
    X2 = np.random.randn(n)
    
    # TODO: 生成倾向得分
    # logit(e) = confounding_strength * (X1 + 0.5*X2)
    propensity_logit = confounding_strength * (X1 + 0.5 * X2)
    propensity = None  # 👈 你的代码: 1 / (1 + np.exp(-propensity_logit))
    
    # TODO: 生成处理
    T = None  # 👈 你的代码: np.random.binomial(1, propensity)
    
    # TODO: 生成结果
    # Y = 5 + 2*T + 1.5*X1 + X2 + noise
    true_ate = 2.0
    Y = None  # 👈 你的代码
    
    df = pd.DataFrame({
        'X1': X1, 'X2': X2, 'T': T, 'Y': Y,
        'true_propensity': propensity
    })
    
    return df, true_ate

<details><summary>💡 点击查看参考答案</summary>

```python
def generate_ipw_data(n: int = 2000, confounding_strength: float = 1.5, seed: int = 42):
    """
    生成用于 IPW 实验的数据
    
    真实 ATE = 2
    """
    np.random.seed(seed)
    
    # 生成协变量
    X1 = np.random.randn(n)
    X2 = np.random.randn(n)
    
    # 生成倾向得分
    propensity_logit = confounding_strength * (X1 + 0.5 * X2)
    propensity = 1 / (1 + np.exp(-propensity_logit))
    
    # 生成处理
    T = np.random.binomial(1, propensity)
    
    # 生成结果
    # Y = 5 + 2*T + 1.5*X1 + X2 + noise
    true_ate = 2.0
    Y = 5 + true_ate * T + 1.5 * X1 + X2 + np.random.randn(n) * 0.5
    
    df = pd.DataFrame({
        'X1': X1, 'X2': X2, 'T': T, 'Y': Y,
        'true_propensity': propensity
    })
    
    return df, true_ate
```

</details>

In [ ]:
def compute_ipw_weights(propensity: np.ndarray, treatment: np.ndarray) -> np.ndarray:
    """
    计算 IPW 权重
    
    权重公式:
    - 处理组: w = 1 / e(X)
    - 控制组: w = 1 / (1 - e(X))
    
    完整公式: w = T/e + (1-T)/(1-e)
    """
    # TODO: 裁剪倾向得分以避免极端权重
    # 将倾向得分限制在 [0.01, 0.99] 范围内
    propensity_clipped = None  # 👈 你的代码: np.clip(propensity, 0.01, 0.99)
    
    # TODO: 计算权重
    # w = T/e + (1-T)/(1-e)
    weights = None  # 👈 你的代码
    
    return weights

<details><summary>💡 点击查看参考答案</summary>

```python
def compute_ipw_weights(propensity: np.ndarray, treatment: np.ndarray) -> np.ndarray:
    """
    计算 IPW 权重
    """
    # 裁剪倾向得分以避免极端权重
    propensity_clipped = np.clip(propensity, 0.01, 0.99)
    
    # 计算权重
    weights = (treatment / propensity_clipped + 
              (1 - treatment) / (1 - propensity_clipped))
    
    return weights
```

</details>

In [ ]:
def estimate_ate_ipw(
    Y: np.ndarray,
    treatment: np.ndarray,
    weights: np.ndarray
) -> Tuple[float, float]:
    """
    使用 IPW 估计 ATE
    
    使用 Hajek 估计器（归一化权重）:
    E[Y(1)] = Σ(Y * w * T) / Σ(w * T)
    E[Y(0)] = Σ(Y * w * (1-T)) / Σ(w * (1-T))
    """
    treated_mask = treatment == 1
    control_mask = treatment == 0
    
    # TODO: 计算加权的 E[Y(1)]
    # 分子: sum(Y * weight) for treated
    # 分母: sum(weight) for treated
    y1_weighted = None  # 👈 你的代码
    
    # TODO: 计算加权的 E[Y(0)]
    y0_weighted = None  # 👈 你的代码
    
    # TODO: 计算 ATE
    ate = None  # 👈 你的代码: y1_weighted - y0_weighted
    
    if ate is None:
        return None, None
    
    # 计算标准误（影响函数方法）
    n = len(Y)
    influence_1 = np.zeros(n)
    influence_1[treated_mask] = (Y[treated_mask] - y1_weighted) * weights[treated_mask]
    
    influence_0 = np.zeros(n)
    influence_0[control_mask] = (Y[control_mask] - y0_weighted) * weights[control_mask]
    
    influence = influence_1 - influence_0
    se = np.sqrt(np.var(influence) / n)
    
    return ate, se

<details><summary>💡 点击查看参考答案</summary>

```python
def estimate_ate_ipw(
    Y: np.ndarray,
    treatment: np.ndarray,
    weights: np.ndarray
) -> Tuple[float, float]:
    """
    使用 IPW 估计 ATE (Hajek 估计器)
    """
    treated_mask = treatment == 1
    control_mask = treatment == 0
    
    # 计算加权的 E[Y(1)]
    y1_weighted = (Y[treated_mask] * weights[treated_mask]).sum() / weights[treated_mask].sum()
    
    # 计算加权的 E[Y(0)]
    y0_weighted = (Y[control_mask] * weights[control_mask]).sum() / weights[control_mask].sum()
    
    # 计算 ATE
    ate = y1_weighted - y0_weighted
    
    # 计算标准误（影响函数方法）
    n = len(Y)
    influence_1 = np.zeros(n)
    influence_1[treated_mask] = (Y[treated_mask] - y1_weighted) * weights[treated_mask]
    
    influence_0 = np.zeros(n)
    influence_0[control_mask] = (Y[control_mask] - y0_weighted) * weights[control_mask]
    
    influence = influence_1 - influence_0
    se = np.sqrt(np.var(influence) / n)
    
    return ate, se
```

</details>

In [ ]:
# 测试 IPW
df, true_ate = generate_ipw_data(n=2000, confounding_strength=1.5)

if df['Y'] is not None:
    X = df[['X1', 'X2']].values
    T = df['T'].values
    Y = df['Y'].values
    
    # 朴素估计
    naive_ate = df[df['T']==1]['Y'].mean() - df[df['T']==0]['Y'].mean()
    print(f"📊 数据概览:")
    print(f"   样本量: {len(df)}")
    print(f"   处理组: {T.sum()} ({T.mean()*100:.1f}%)")
    print(f"   真实 ATE: {true_ate}")
    print(f"   朴素 ATE: {naive_ate:.4f} (偏差: {naive_ate - true_ate:+.4f})")
    
    # 估计倾向得分
    lr = LogisticRegression(max_iter=1000)
    lr.fit(X, T)
    propensity = lr.predict_proba(X)[:, 1]
    
    print(f"\n倾向得分范围: [{propensity.min():.4f}, {propensity.max():.4f}]")
else:
    print("❌ 请完成 generate_ipw_data 函数！")

In [ ]:
# 计算 IPW 权重
if df['Y'] is not None:
    weights = compute_ipw_weights(propensity, T)
    
    if weights is not None:
        print("⚖️ IPW 权重统计:")
        print(f"   范围: [{weights.min():.4f}, {weights.max():.4f}]")
        print(f"   均值: {weights.mean():.4f}")
        print(f"   处理组权重均值: {weights[T==1].mean():.4f}")
        print(f"   控制组权重均值: {weights[T==0].mean():.4f}")
        
        # 计算 IPW ATE
        ipw_ate, ipw_se = estimate_ate_ipw(Y, T, weights)
        
        if ipw_ate is not None:
            print(f"\n📈 IPW ATE 估计:")
            print(f"   IPW ATE: {ipw_ate:.4f} ± {ipw_se:.4f}")
            print(f"   95% CI: [{ipw_ate - 1.96*ipw_se:.4f}, {ipw_ate + 1.96*ipw_se:.4f}]")
            print(f"   偏差: {ipw_ate - true_ate:+.4f}")
            
            print(f"\n🔬 对比:")
            print(f"   朴素估计偏差: {naive_ate - true_ate:+.4f}")
            print(f"   IPW 估计偏差: {ipw_ate - true_ate:+.4f}")
            
            if abs(ipw_ate - true_ate) < abs(naive_ate - true_ate):
                print(f"   ✅ IPW 减少了 {abs(naive_ate - true_ate) - abs(ipw_ate - true_ate):.2f} 的偏差！")
        else:
            print("❌ 请完成 estimate_ate_ipw 函数！")
    else:
        print("❌ 请完成 compute_ipw_weights 函数！")

---

## 🔍 Part 3: 权重诊断

### 极端权重的问题

当倾向得分接近 0 或 1 时，权重会变得非常大，导致：
- 估计不稳定（方差大）
- 少数样本主导结果

### 有效样本量 (ESS)

ESS 衡量「有多少样本真正在起作用」：

$$ESS = \frac{(\sum w_i)^2}{\sum w_i^2}$$

- 当所有权重相等时，ESS = n
- 当权重差异很大时，ESS << n

In [ ]:
def compute_effective_sample_size(weights: np.ndarray) -> float:
    """
    计算有效样本量
    
    ESS = (sum(w))^2 / sum(w^2)
    """
    # 👈 你的代码
    ess = None  # (weights.sum())**2 / (weights**2).sum()
    return ess


def diagnose_weights(weights: np.ndarray, treatment: np.ndarray) -> dict:
    """
    诊断 IPW 权重分布
    """
    treated_weights = weights[treatment == 1]
    control_weights = weights[treatment == 0]
    
    # TODO: 计算有效样本量
    ess = compute_effective_sample_size(weights)
    ess_fraction = ess / len(weights) if ess else None
    
    stats = {
        'n_samples': len(weights),
        'ess': ess,
        'ess_fraction': ess_fraction,
        'treated_weight_mean': treated_weights.mean(),
        'treated_weight_max': treated_weights.max(),
        'control_weight_mean': control_weights.mean(),
        'control_weight_max': control_weights.max(),
        'max_weight': weights.max(),
        'weight_cv': weights.std() / weights.mean()  # coefficient of variation
    }
    
    return stats

<details><summary>💡 点击查看参考答案</summary>

```python
def compute_effective_sample_size(weights: np.ndarray) -> float:
    """
    计算有效样本量
    
    ESS = (sum(w))^2 / sum(w^2)
    """
    ess = (weights.sum())**2 / (weights**2).sum()
    return ess
```

</details>

In [ ]:
# 权重诊断
if weights is not None:
    diag = diagnose_weights(weights, T)
    
    if diag['ess'] is not None:
        print("🔍 权重诊断报告:")
        print("=" * 50)
        print(f"总样本量: {diag['n_samples']}")
        print(f"有效样本量 (ESS): {diag['ess']:.1f} ({diag['ess_fraction']*100:.1f}%)")
        print(f"\n处理组权重:")
        print(f"   均值: {diag['treated_weight_mean']:.4f}")
        print(f"   最大: {diag['treated_weight_max']:.4f}")
        print(f"\n控制组权重:")
        print(f"   均值: {diag['control_weight_mean']:.4f}")
        print(f"   最大: {diag['control_weight_max']:.4f}")
        print(f"\n权重变异系数 (CV): {diag['weight_cv']:.4f}")
        
        if diag['ess_fraction'] < 0.5:
            print("\n⚠️ 警告: ESS 小于 50%，可能有极端权重问题！")
        else:
            print("\n✅ 权重分布良好")
        
        # 可视化权重分布
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        ax1 = axes[0]
        ax1.hist(weights[T==1], bins=30, alpha=0.5, label='处理组', color='coral')
        ax1.hist(weights[T==0], bins=30, alpha=0.5, label='控制组', color='steelblue')
        ax1.set_xlabel('权重', fontsize=12)
        ax1.set_ylabel('频数', fontsize=12)
        ax1.set_title('IPW 权重分布', fontsize=14)
        ax1.legend()
        
        ax2 = axes[1]
        ax2.scatter(propensity[T==1], weights[T==1], alpha=0.3, label='处理组', c='coral')
        ax2.scatter(propensity[T==0], weights[T==0], alpha=0.3, label='控制组', c='steelblue')
        ax2.set_xlabel('倾向得分', fontsize=12)
        ax2.set_ylabel('权重', fontsize=12)
        ax2.set_title('权重 vs 倾向得分', fontsize=14)
        ax2.legend()
        
        plt.tight_layout()
        plt.show()
    else:
        print("❌ 请完成 compute_effective_sample_size 函数！")

---

## ✂️ Part 4: 权重裁剪与稳定权重

### 方法 1: 裁剪极端权重

将权重限制在某个百分位数以下：

$$w_i^{\text{clipped}} = \min(w_i, \text{percentile}_{99})$$

### 方法 2: 稳定权重 (Stabilized Weights)

在分子上乘以边际处理概率：

$$w_i^{\text{stab}} = \frac{P(T)}{e(X_i)} \text{ for treated}, \quad \frac{1-P(T)}{1-e(X_i)} \text{ for control}$$

稳定权重的均值接近 1，方差更小。

In [ ]:
def clip_extreme_weights(weights: np.ndarray, percentile: float = 99) -> np.ndarray:
    """
    裁剪极端权重
    """
    # TODO: 计算权重的百分位数
    max_weight = None  # 👈 你的代码: np.percentile(weights, percentile)
    
    # TODO: 裁剪权重
    clipped = None  # 👈 你的代码: np.clip(weights, None, max_weight)
    
    return clipped


def compute_stabilized_weights(propensity: np.ndarray, treatment: np.ndarray) -> np.ndarray:
    """
    计算稳定权重
    
    w_stab = P(T) / e(X) for treated
    w_stab = (1-P(T)) / (1-e(X)) for control
    """
    # TODO: 计算边际处理概率
    marginal_prob = None  # 👈 你的代码: treatment.mean()
    
    # TODO: 裁剪倾向得分
    propensity_clipped = np.clip(propensity, 0.01, 0.99)
    
    # TODO: 计算稳定权重
    weights_stab = np.zeros(len(treatment))
    
    # 处理组: w = marginal_prob / propensity
    # 👈 你的代码
    
    # 控制组: w = (1 - marginal_prob) / (1 - propensity)
    # 👈 你的代码
    
    return weights_stab

<details><summary>💡 点击查看参考答案</summary>

```python
def clip_extreme_weights(weights: np.ndarray, percentile: float = 99) -> np.ndarray:
    """
    裁剪极端权重
    """
    # 计算权重的百分位数
    max_weight = np.percentile(weights, percentile)
    
    # 裁剪权重
    clipped = np.clip(weights, None, max_weight)
    
    return clipped


def compute_stabilized_weights(propensity: np.ndarray, treatment: np.ndarray) -> np.ndarray:
    """
    计算稳定权重
    """
    # 计算边际处理概率
    marginal_prob = treatment.mean()
    
    # 裁剪倾向得分
    propensity_clipped = np.clip(propensity, 0.01, 0.99)
    
    # 计算稳定权重
    weights_stab = np.zeros(len(treatment))
    
    # 处理组: w = marginal_prob / propensity
    weights_stab[treatment == 1] = marginal_prob / propensity_clipped[treatment == 1]
    
    # 控制组: w = (1 - marginal_prob) / (1 - propensity)
    weights_stab[treatment == 0] = (1 - marginal_prob) / (1 - propensity_clipped[treatment == 0])
    
    return weights_stab
```

</details>

In [ ]:
# 比较不同权重方法
if weights is not None:
    # 裁剪权重
    weights_clipped = clip_extreme_weights(weights, 99)
    
    # 稳定权重
    weights_stab = compute_stabilized_weights(propensity, T)
    
    # 对比
    results = []
    
    for name, w in [('标准 IPW', weights), 
                    ('裁剪权重 (99%)', weights_clipped),
                    ('稳定权重', weights_stab)]:
        if w is not None and w.sum() > 0:
            ate, se = estimate_ate_ipw(Y, T, w)
            ess = compute_effective_sample_size(w)
            
            results.append({
                '方法': name,
                'ATE': ate,
                'SE': se,
                '偏差': ate - true_ate if ate else None,
                'ESS': ess,
                'Max Weight': w.max()
            })
    
    if results:
        results_df = pd.DataFrame(results)
        print("🔬 不同权重方法的比较:")
        print("=" * 70)
        display(results_df.round(4))
        
        print("\n💡 观察:")
        print("   - 稳定权重和裁剪权重可以减少权重方差")
        print("   - ESS 越高，估计越稳定")
        print("   - 但过度裁剪可能引入偏差")

---

## 📈 Part 5: 混淆强度实验

让我们看看在不同混淆强度下，IPW 的表现如何。

In [ ]:
def run_ipw_experiment(confounding_strengths: list = [0.5, 1.0, 1.5, 2.0, 2.5]):
    """
    在不同混淆强度下测试 IPW
    """
    results = []
    
    for strength in confounding_strengths:
        df, true_ate = generate_ipw_data(n=2000, confounding_strength=strength)
        
        if df['Y'] is None:
            continue
            
        X = df[['X1', 'X2']].values
        T = df['T'].values
        Y = df['Y'].values
        
        # 朴素估计
        naive_ate = df[df['T']==1]['Y'].mean() - df[df['T']==0]['Y'].mean()
        
        # 估计倾向得分
        lr = LogisticRegression(max_iter=1000)
        lr.fit(X, T)
        propensity = lr.predict_proba(X)[:, 1]
        
        # IPW 估计
        weights = compute_ipw_weights(propensity, T)
        if weights is not None:
            ipw_ate, ipw_se = estimate_ate_ipw(Y, T, weights)
            ess = compute_effective_sample_size(weights)
        else:
            ipw_ate, ipw_se, ess = None, None, None
        
        results.append({
            '混淆强度': strength,
            '真实 ATE': true_ate,
            '朴素 ATE': naive_ate,
            '朴素偏差': naive_ate - true_ate,
            'IPW ATE': ipw_ate,
            'IPW 偏差': ipw_ate - true_ate if ipw_ate else None,
            'ESS': ess
        })
    
    return pd.DataFrame(results)

In [ ]:
# 运行实验
exp_results = run_ipw_experiment()

if not exp_results.empty and exp_results['IPW ATE'].notna().any():
    print("📊 混淆强度实验结果:")
    print("=" * 80)
    display(exp_results.round(4))
    
    # 可视化
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1 = axes[0]
    ax1.plot(exp_results['混淆强度'], exp_results['朴素 ATE'], 
             'o-', label='朴素估计', color='red', markersize=8)
    ax1.plot(exp_results['混淆强度'], exp_results['IPW ATE'], 
             's-', label='IPW 估计', color='green', markersize=8)
    ax1.axhline(2.0, color='blue', linestyle='--', label='真实 ATE', linewidth=2)
    ax1.set_xlabel('混淆强度', fontsize=12)
    ax1.set_ylabel('ATE 估计', fontsize=12)
    ax1.set_title('ATE 估计 vs 混淆强度', fontsize=14)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    ax2 = axes[1]
    ax2.bar(exp_results['混淆强度'] - 0.1, np.abs(exp_results['朴素偏差']), 
            width=0.2, label='朴素估计', color='red', alpha=0.7)
    ax2.bar(exp_results['混淆强度'] + 0.1, np.abs(exp_results['IPW 偏差']), 
            width=0.2, label='IPW 估计', color='green', alpha=0.7)
    ax2.set_xlabel('混淆强度', fontsize=12)
    ax2.set_ylabel('|偏差|', fontsize=12)
    ax2.set_title('偏差绝对值 vs 混淆强度', fontsize=14)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print("\n💡 关键发现:")
    print("   1. IPW 在各种混淆强度下都能有效减少偏差")
    print("   2. 混淆强度越大，朴素估计偏差越大")
    print("   3. 但混淆强度大时，ESS 会下降（极端权重增多）")
else:
    print("请先完成前面的 TODO 部分！")

---

## 📝 Part 6: 思考题

### 问题 1: IPW 的核心思想是什么？为什么重加权可以去除混淆偏差？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 2: IPW 权重的含义是什么？为什么处理组用 1/e(X)，控制组用 1/(1-e(X))？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 3: 什么情况下会出现极端权重？极端权重有什么问题？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 4: 有效样本量 (ESS) 的含义是什么？ESS 小意味着什么？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 5: IPW 相比 PSM 有什么优缺点？

**你的答案:**

*（在这里写下你的思考...）*

---

### 问题 6: 如果倾向得分模型误设定，IPW 估计会怎样？

**提示:** 这引出了下一节的主题——双重稳健估计

**你的答案:**

*（在这里写下你的思考...）*

---

## 💻 从零实现 IPW

让我们实现一个完整的 IPW 估计器类！

In [ ]:
class MyIPWEstimator:
    """
    从零实现的 IPW 估计器
    
    功能:
    - 估计倾向得分
    - 计算 IPW 权重（标准/稳定/裁剪）
    - 估计 ATE 和标准误
    - 诊断权重分布
    """
    
    def __init__(self, trim_propensity: Tuple[float, float] = (0.01, 0.99),
                 stabilize: bool = False,
                 clip_weights_percentile: Optional[float] = None):
        """
        Args:
            trim_propensity: 倾向得分裁剪范围
            stabilize: 是否使用稳定权重
            clip_weights_percentile: 权重裁剪百分位数（如 99）
        """
        self.trim_propensity = trim_propensity
        self.stabilize = stabilize
        self.clip_weights_percentile = clip_weights_percentile
        self.propensity_ = None
        self.weights_ = None
        
    def fit(self, X: np.ndarray, T: np.ndarray, Y: np.ndarray):
        """拟合 IPW 模型"""
        self.X_ = X
        self.T_ = T
        self.Y_ = Y
        
        # 估计倾向得分
        self.propensity_model_ = LogisticRegression(max_iter=1000)
        self.propensity_model_.fit(X, T)
        self.propensity_ = self.propensity_model_.predict_proba(X)[:, 1]
        
        # 裁剪倾向得分
        self.propensity_ = np.clip(self.propensity_, 
                                   self.trim_propensity[0], 
                                   self.trim_propensity[1])
        
        # 计算权重
        if self.stabilize:
            self.weights_ = self._compute_stabilized_weights()
        else:
            self.weights_ = self._compute_weights()
        
        # 裁剪权重
        if self.clip_weights_percentile is not None:
            max_w = np.percentile(self.weights_, self.clip_weights_percentile)
            self.weights_ = np.clip(self.weights_, None, max_w)
        
        return self
    
    def _compute_weights(self):
        """计算标准 IPW 权重"""
        weights = (self.T_ / self.propensity_ + 
                  (1 - self.T_) / (1 - self.propensity_))
        return weights
    
    def _compute_stabilized_weights(self):
        """计算稳定权重"""
        marginal_prob = self.T_.mean()
        weights = np.zeros(len(self.T_))
        weights[self.T_ == 1] = marginal_prob / self.propensity_[self.T_ == 1]
        weights[self.T_ == 0] = (1 - marginal_prob) / (1 - self.propensity_[self.T_ == 0])
        return weights
    
    def estimate_ate(self) -> Tuple[float, float]:
        """估计 ATE（Hajek 估计器）"""
        if self.weights_ is None:
            raise ValueError("Must fit before estimating ATE")
        
        treated_mask = self.T_ == 1
        control_mask = self.T_ == 0
        
        # Hajek 估计器（归一化权重）
        y1_hat = (self.Y_[treated_mask] * self.weights_[treated_mask]).sum() / \
                 self.weights_[treated_mask].sum()
        
        y0_hat = (self.Y_[control_mask] * self.weights_[control_mask]).sum() / \
                 self.weights_[control_mask].sum()
        
        ate = y1_hat - y0_hat
        
        # 计算标准误（基于影响函数）
        n = len(self.Y_)
        influence_1 = np.zeros(n)
        influence_1[treated_mask] = (self.Y_[treated_mask] - y1_hat) * \
                                    self.weights_[treated_mask] / self.weights_[treated_mask].sum()
        
        influence_0 = np.zeros(n)
        influence_0[control_mask] = (self.Y_[control_mask] - y0_hat) * \
                                    self.weights_[control_mask] / self.weights_[control_mask].sum()
        
        influence = influence_1 - influence_0
        se = np.sqrt((influence ** 2).sum()) / np.sqrt(n)
        
        return ate, se
    
    def compute_ess(self) -> float:
        """计算有效样本量"""
        if self.weights_ is None:
            raise ValueError("Must fit before computing ESS")
        
        return (self.weights_.sum() ** 2) / (self.weights_ ** 2).sum()
    
    def diagnose(self) -> Dict:
        """诊断权重分布"""
        if self.weights_ is None:
            raise ValueError("Must fit before diagnosing")
        
        treated_weights = self.weights_[self.T_ == 1]
        control_weights = self.weights_[self.T_ == 0]
        
        return {
            'n_samples': len(self.Y_),
            'ess': self.compute_ess(),
            'ess_fraction': self.compute_ess() / len(self.Y_),
            'weight_mean': self.weights_.mean(),
            'weight_std': self.weights_.std(),
            'weight_min': self.weights_.min(),
            'weight_max': self.weights_.max(),
            'weight_cv': self.weights_.std() / self.weights_.mean(),
            'treated_weight_mean': treated_weights.mean(),
            'treated_weight_max': treated_weights.max(),
            'control_weight_mean': control_weights.mean(),
            'control_weight_max': control_weights.max(),
        }
    
    def summary(self):
        """打印估计结果摘要"""
        ate, se = self.estimate_ate()
        diag = self.diagnose()
        
        print("=" * 60)
        print("IPW 估计结果")
        print("=" * 60)
        print(f"ATE 估计: {ate:.4f} ± {se:.4f}")
        print(f"95% CI: [{ate - 1.96*se:.4f}, {ate + 1.96*se:.4f}]")
        print()
        print(f"样本量: {diag['n_samples']}")
        print(f"有效样本量 (ESS): {diag['ess']:.1f} ({diag['ess_fraction']*100:.1f}%)")
        print()
        print(f"权重统计:")
        print(f"  范围: [{diag['weight_min']:.4f}, {diag['weight_max']:.4f}]")
        print(f"  均值: {diag['weight_mean']:.4f}")
        print(f"  标准差: {diag['weight_std']:.4f}")
        print(f"  变异系数: {diag['weight_cv']:.4f}")
        print("=" * 60)


# 测试我们的实现
print("🧪 测试自己实现的 IPW 估计器\n")

# 生成数据
np.random.seed(42)
n = 1500
X_test = np.random.randn(n, 3)
propensity_logit = 1.5 * (X_test[:, 0] + 0.5 * X_test[:, 1])
propensity_true = 1 / (1 + np.exp(-propensity_logit))
T_test = np.random.binomial(1, propensity_true)
Y_test = 2.0 * T_test + 1.5 * X_test[:, 0] + X_test[:, 1] + np.random.randn(n) * 0.5

print(f"真实 ATE = 2.0\n")

# 测试三种配置
configs = [
    {"name": "标准 IPW", "params": {}},
    {"name": "稳定权重 IPW", "params": {"stabilize": True}},
    {"name": "裁剪权重 IPW (99%)", "params": {"clip_weights_percentile": 99}}
]

for config in configs:
    print(f"\n{'='*60}")
    print(f"配置: {config['name']}")
    print(f"{'='*60}")
    ipw = MyIPWEstimator(**config['params'])
    ipw.fit(X_test, T_test, Y_test)
    ipw.summary()

---

## 🎤 高频面试题

### 面试题 1: IPW 的核心思想是什么？为什么重加权可以去除混淆偏差？

**参考回答**:

IPW 的核心思想是通过**重新加权**，创造一个"伪总体"，在这个伪总体中处理是随机分配的。

**直观解释**:
- 在观测数据中，某些类型的人更可能接受处理（比如年轻人更可能用新药）
- 这导致处理组和控制组的人群不可比
- IPW 给每个人赋予权重：$w_i = \frac{T_i}{e(X_i)} + \frac{1-T_i}{1-e(X_i)}$
- 这个权重让"不太可能接受处理但接受了"的人贡献更大
- 加权后的数据就像随机实验一样，X 与 T 独立了！

**数学本质**: IPW 是 Horvitz-Thompson 估计量，通过逆概率加权来纠正选择偏差。

---

### 面试题 2: IPW 的局限性是什么？如何解决？

**参考回答**:

**主要局限性**:

1. **极端权重问题**
   - 当 $e(X) \approx 0$ 或 $e(X) \approx 1$ 时，权重 $\frac{1}{e(X)}$ 或 $\frac{1}{1-e(X)}$ 会非常大
   - 少数几个样本主导结果，估计不稳定，方差巨大
   
2. **依赖倾向得分模型**
   - 如果倾向得分模型误设定，IPW 估计有偏
   - 没有"双重稳健"性质

3. **效率损失**
   - 相比结果模型方法，方差可能更大

**解决方法**:

1. **权重裁剪 (Weight Trimming)**
   ```python
   max_weight = np.percentile(weights, 99)
   weights_clipped = np.clip(weights, None, max_weight)
   ```

2. **稳定权重 (Stabilized Weights)**
   $$w_i^{stab} = \frac{P(T=T_i)}{P(T=T_i|X_i)}$$
   稳定权重的均值接近 1，方差更小

3. **修剪倾向得分 (Propensity Score Trimming)**
   - 丢弃倾向得分过于极端的样本（如 <0.1 或 >0.9）

4. **使用双重稳健方法 (AIPW)**
   - 结合 IPW 和结果模型，更稳健

---

### 面试题 3: 什么是有效样本量(ESS)？它的意义是什么？

**参考回答**:

**定义**:
$$ESS = \frac{(\sum w_i)^2}{\sum w_i^2}$$

**意义**:
- ESS 衡量"有多少样本在真正起作用"
- 当所有权重相等时，ESS = n（所有样本都有效）
- 当权重差异很大时，ESS << n（少数样本主导）

**例子**:
- 如果 n=1000，但 ESS=100，说明实际上只有 100 个样本的信息量
- 标准误应该用 ESS 而不是 n 来计算

**经验法则**:
- ESS / n > 0.5: 良好
- ESS / n < 0.3: 警告，可能需要修剪极端权重

---

### 面试题 4: IPW 相比 PSM 有什么优缺点？

**参考回答**:

| 方面 | PSM | IPW |
|------|-----|-----|
| **样本利用** | 丢弃未匹配样本 | 使用所有样本 ✅ |
| **估计量** | ATT | ATE ✅ |
| **极端值** | 自动丢弃 ✅ | 需要处理极端权重 ❌ |
| **直观性** | 很直观 ✅ | 稍抽象 ❌ |
| **方差** | 通常较小 ✅ | 可能较大 ❌ |
| **平衡检查** | 易于检查 ✅ | 需要加权平衡检查 ❌ |

**选择建议**:
- 小数据、需要可解释性 → PSM
- 大数据、需要 ATE、担心样本损失 → IPW
- 高维数据、担心模型误设定 → AIPW（双重稳健）

---

---

## 🎉 总结

### 核心概念

| 概念 | 公式 | 作用 |
|-----|------|------|
| IPW 权重 | $w = T/e + (1-T)/(1-e)$ | 创造伪总体 |
| 有效样本量 | $ESS = (\sum w)^2 / \sum w^2$ | 衡量权重质量 |
| 稳定权重 | $w^{stab} = P(T)/e$ | 减少权重方差 |

### IPW vs PSM

| 方面 | PSM | IPW |
|-----|-----|-----|
| 样本利用 | 丢弃未匹配样本 | 使用所有样本 |
| 估计量 | ATT | ATE |
| 极端值处理 | 自动丢弃 | 需要裁剪权重 |
| 直观性 | 很直观 | 略抽象 |

### 下一步

IPW 依赖于倾向得分模型的正确性。如果模型错了怎么办？

下一节我们将学习**双重稳健估计 (AIPW)**——一种只要倾向得分模型或结果模型之一正确就能得到无偏估计的方法！

---

**「给每个人恰当的权重，让数据告诉我们真相。」**